In [1]:
%pylab
%matplotlib inline
%load_ext watermark
%watermark -v -p sklearn,konlpy,pandas

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib
CPython 3.5.4
IPython 6.1.0

sklearn 0.19.1
konlpy 0.4.4
pandas 0.21.0


In [2]:
import konlpy
import pandas as pd

데이터 파일을 읽어 리뷰 텍스트와 점수를 text_train, y_train 변수에 저장합니다. 데이터 파일의 내용은 번호, 텍스트, 레이블이 탭으로 구분되어 한 라인에 한개의 데이터 샘플이 들어 있습니다.

In [3]:
df_train = pd.read_csv('data/ratings_train.txt', delimiter='\t', keep_default_na=False)

In [4]:
df_train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [5]:
text_train, y_train = df_train['document'].as_matrix(), df_train['label'].as_matrix()

같은 방식으로 테스트 데이터를 읽습니다.

In [6]:
df_test = pd.read_csv('data/ratings_test.txt', delimiter='\t', keep_default_na=False)
text_test = df_test['document'].as_matrix()
y_test = df_test['label'].as_matrix()

훈련 데이터와 테스트 데이터의 크기를 확인합니다.

In [7]:
len(text_train), np.bincount(y_train)

(150000, array([75173, 74827]))

In [8]:
len(text_test), np.bincount(y_test)

(50000, array([24827, 25173]))

KoNLPy의 `Twitter` 클래스 개체를 만듭니다

In [9]:
from konlpy.tag import Twitter
twitter_tag = Twitter()

`TfidfVectorizer` 의 토큰 파서를 대체하기 위해 `Twitter` 클래스를 사용하는 함수를 만듭니다.

In [10]:
def twitter_tokenizer(text):
    return twitter_tag.morphs(text)

`min_df`, `ngram_range`, `C` 매개변수를 대상으로 그리드 서치를 수행합니다.

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

twit_param_grid = {'tfidfvectorizer__min_df': [3, 5 ,7],
              'tfidfvectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)],
              'logisticregression__C': [0.1, 1, 10]}
twit_pipe = make_pipeline(TfidfVectorizer(tokenizer=twitter_tokenizer), LogisticRegression())
twit_grid = GridSearchCV(twit_pipe, twit_param_grid)

# 그리드 서치를 수행합니다
twit_grid.fit(text_train[0:1000], y_train[0:1000])
print("최상의 크로스 밸리데이션 점수: {:.3f}".format(twit_grid.best_score_))
print("최적의 크로스 밸리데이션 파라미터: ", twit_grid.best_params_)

최상의 크로스 밸리데이션 점수: 0.727
최적의 크로스 밸리데이션 파라미터:  {'tfidfvectorizer__ngram_range': (1, 1), 'tfidfvectorizer__min_df': 3, 'logisticregression__C': 1}


In [12]:
X_test_twit = twit_grid.best_estimator_.named_steps["tfidfvectorizer"].transform(text_test)
score = twit_grid.best_estimator_.named_steps["logisticregression"].score(X_test_twit, y_test)
print("테스트 세트 점수: {:.3f}".format(score))

테스트 세트 점수: 0.721


병렬 처리를 위해 `Mecab`을 사용하여 전체 데이터로 학습시킵니다. `Mecab`으로 토큰을 분할하는 함수를 만듭니다.

In [13]:
from konlpy.tag import Mecab
mecab = Mecab()
def mecab_tokenizer(text):
    return mecab.morphs(text)

규제 파라미터의 범위를 확대하여 그리드 서치를 수행합니다.

In [14]:
mecab_param_grid = {'tfidfvectorizer__min_df': [3, 5 ,7],
              'tfidfvectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)],
              'logisticregression__C': [0.1, 1, 10, 100]}
mecab_pipe = make_pipeline(TfidfVectorizer(tokenizer=mecab_tokenizer), LogisticRegression())
mecab_grid = GridSearchCV(mecab_pipe, mecab_param_grid)

# 그리드 서치를 수행합니다
mecab_grid.fit(text_train, y_train)
print("최상의 크로스 밸리데이션 점수: {:.3f}".format(mecab_grid.best_score_))
print("최적의 크로스 밸리데이션 파라미터: ", mecab_grid.best_params_)

최상의 크로스 밸리데이션 점수: 0.869
최적의 크로스 밸리데이션 파라미터:  {'tfidfvectorizer__ngram_range': (1, 3), 'tfidfvectorizer__min_df': 3, 'logisticregression__C': 10}


In [15]:
X_test_mecab = mecab_grid.best_estimator_.named_steps["tfidfvectorizer"].transform(text_test)
score = mecab_grid.best_estimator_.named_steps["logisticregression"].score(X_test_mecab, y_test)
print("테스트 세트 점수: {:.3f}".format(score))

테스트 세트 점수: 0.875
